# Classification Hackathon



<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>


 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    

In [ ]:
#libraries for...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import pickle
import re
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SV

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>


In [ ]:
train_df= pd.read_csv('resources/train_set.csv')
test_df= pd.read_csv('resources/test_set.csv')

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>



In [ ]:
# to get a visaual assessment of different sampled observations in the train dataset
train_df.sample(20)

In [ ]:
# to get a visaual assessment of different sampled observations in the test dataset
test_df.sample(20)

In [ ]:
# Futher look at the some smapled contents of contents of the text each dataset
# Train Data
for i in train_df['text'].sample(10):
    print(i, '\n')

#### Obvervations
##### Visual Assessment
1. The train contains the text and language id, while the test contains only text
2. All text appear to be in lower case
3. there still appeaers to be a bit of hyphens in the some texts

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>



In [ ]:
# text cleaning...

In [1]:
def messages_processing(text):
    """
    Returns a sringof words, tokeninses and removes noise in the string
    
    Parameters:
        text (str): the string to be tokenised.
    Returns:
        text (str): returns a string of words with no punctuations
    """
    text=re.sub('<.*?>', '', text)
    text = re.sub("\n","",text)
    text = text.lower()
    text=' '.join(text.split())
   
    return text

In [ ]:
train_df.text= train_df.text.apply(messages_processing)
test_df.text= test_df.text.apply(messages_processing)

In [ ]:
# to create features

vect = CountVectorizer(ngram_range=(1, 10), analyzer='char').fit(train_df['text'])
X_counts= vect.transform(train_df['text'])
pred_x= vect.transform(test_df['text'])

In [ ]:
# To scale both test and train features

trans= TfidfVectorizer(norm='l1', sublinear_tf=True).fit(train_df['text'])
X_tran= trans.transform(X_counts)
pred_x= trans.transform(pred_x)
norm= MaxAbsScaler(copy=False)
X= norm.fit_transform(X_tran)

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>


In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
# create one or more ML models
model = MultinomialNB(alpha=1, fit_prior=True, )
model.fit(X_test,y_test)

model.score(X_test,y_test)

In [ ]:
#hyperparameter tuning

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>


In [ ]:
# Compare model performance
y_pred= model.predict(X_test)
logreg_acc= accuracy_score(y_pred, y_test)
print(f"Test accuracy: {logreg_acc}"

In [ ]:
# Classification report

def get_class_report(clf, X_test, y_test):
    #your code here
    y_pred= clf.predict(X_test)
    report= classification_report(y_test, y_pred)
    return report

print(get_class_report(model, X_test, y_test))

In [ ]:
# Predict test

y_pred= model.predict(pred_x)

In [ ]:
# Create submission dataframe

a=len(y_pred)
b= [i for i in range(1, a+1)]
sub_df= pd.DataFrame({'index': b, 'lang_id': y_pred})
sub_df.to_csv('mnb2.csv', index=False)